DEFINE "PATH_TO_IMAGES" on your local machine

In [1]:
image_size = 384
seed = 42
batch_size = 1
num_workers = 24

kernel_type = 'vit_base_r50_s16_384'
data_dir = PATH_TO_IMAGES # !!! DEFINE "PATH_TO_IMAGES" on your local machine
model_dir = f'weights/'

In [2]:
import pandas as pd
import numpy as np
import sys
import os
import time
import cv2
import PIL.Image
import random
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR 
from warmup_scheduler import GradualWarmupScheduler
import albumentations as A
from albumentations import *
from tqdm import tqdm
import matplotlib.pyplot as plt
import gc
from sklearn.metrics import roc_auc_score
import seaborn as sns
from pylab import rcParams
import timm
from warnings import filterwarnings
from albumentations.pytorch import ToTensorV2
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts

from warnings import filterwarnings
filterwarnings("ignore")

In [3]:
class CovidGeneralModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=4, in_chans=3)

    def forward(self, x):
        output = self.model(x)
        return output

In [4]:
class CovidDataset(Dataset):
    def __init__(self, df, mode, transform=None):
        
        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.loc[index]
        image_id = row.image_id
        img = cv2.imread(row.file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            res = self.transform(image=img)
            img = res['image']
                
        img = img.type('torch.FloatTensor')
        img = img / 255.0
        return torch.tensor(img).float(), image_id

In [5]:
valid_transform = A.Compose([
    A.Resize(image_size, image_size, p=1.0),
    ToTensorV2()
], p=1.0)
df_train = pd.read_csv('../cross_validation_scheme/crossval_info/original_data/multi_label_problem.csv')
df_train['image_name'] = df_train['image_id'].apply(lambda x: x.replace('_image', '.png'))
df_train['file_path'] = df_train.image_name.apply(lambda x: os.path.join(data_dir, f'{x}'))
dataset = CovidDataset(df_train, 'train', transform=valid_transform)
test_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True
)

In [6]:
def tta_inference_func(model, test_loader):
    result = {}
    model.eval()
    bar = tqdm(test_loader)
    LOGITS = []
    PREDS = []
    IMAGES = []
    
    with torch.no_grad():
        for batch_idx, (images, image_id) in enumerate(bar):
            x = images.to(device)
            x = torch.stack([x,x.flip(-1)],0) # hflip
            x = x.view(-1, 3, image_size, image_size)
            logits = model(x)
            logits = logits.view(batch_size, 2, -1).mean(1)
            PREDS += [logits.sigmoid().detach().cpu()]
            LOGITS.append(logits.cpu())
            IMAGES += image_id
        PREDS = torch.cat(PREDS).cpu().numpy()
    result['image_id'] = IMAGES
    result['negative'] = PREDS[:, 0]
    result['typical'] = PREDS[:, 1]
    result['indeterminate'] = PREDS[:, 2]
    result['atypical'] = PREDS[:, 3]
    df = pd.DataFrame(result)
    return df

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelsEffnet = []
MODELS_EFFNET = [
    'weights/vit_base_r50_s16_384_fold0_best_AP.pth',
    'weights/vit_base_r50_s16_384_fold1_best_AP.pth',
    'weights/vit_base_r50_s16_384_fold2_best_AP.pth',
    'weights/vit_base_r50_s16_384_fold3_best_AP.pth',
    'weights/vit_base_r50_s16_384_fold4_best_AP.pth'
]
for path in MODELS_EFFNET:
    state_dict = torch.load(path, map_location=torch.device('cpu'))
    model = CovidGeneralModel(kernel_type, pretrained=False)
    model.load_state_dict(state_dict)
    model.eval()
    model.to(device)
    modelsEffnet.append(model)
del state_dict

In [8]:
df_list = list()
for model in modelsEffnet:
    df = tta_inference_func(model, test_loader)
    df_list.append(df)

100%|██████████| 6334/6334 [02:39<00:00, 39.59it/s]


In [9]:
df_result = pd.concat(df_list).groupby('image_id').mean()
df_result

,negative,typical,indeterminate,atypical
image_id,,,,
000a312787f2_image,0.011603,0.875717,0.099689,0.013051
000c3a3f293f_image,0.156081,0.431505,0.259723,0.112396
0012ff7358bc_image,0.010057,0.874840,0.101618,0.015602
001398f4ff4f_image,0.719496,0.100763,0.137856,0.043034
001bd15d1891_image,0.023360,0.794622,0.151826,0.028687
...,...,...,...,...
ffcc6edd9445_image,0.081923,0.404266,0.378179,0.130869
ffd91a2c4ca0_image,0.837703,0.062449,0.078890,0.019056
ffd9b6cf2961_image,0.528679,0.165514,0.213177,0.080163


In [10]:
df_result.reset_index(inplace=True)
meta = df_train[['image_id', 'study_id']]
df_result = meta.merge(df_result, on='image_id')
df_result

,image_id,study_id,negative,typical,indeterminate,atypical
0,000a312787f2_image,5776db0cec75_study,0.011603,0.875717,0.099689,0.013051
1,000c3a3f293f_image,ff0879eb20ed_study,0.156081,0.431505,0.259723,0.112396
2,0012ff7358bc_image,9d514ce429a7_study,0.010057,0.874840,0.101618,0.015602
3,001398f4ff4f_image,28dddc8559b2_study,0.719496,0.100763,0.137856,0.043034
4,001bd15d1891_image,dfd9fdd85a3e_study,0.023360,0.794622,0.151826,0.028687
...,...,...,...,...,...,...
6329,ffcc6edd9445_image,7e6c68462e06_study,0.081923,0.404266,0.378179,0.130869
6330,ffd91a2c4ca0_image,8332bdaddb6e_study,0.837703,0.062449,0.078890,0.019056
6331,ffd9b6cf2961_image,7eed9af03814_study,0.528679,0.165514,0.213177,0.080163
6332,ffdc682f7680_image,a0cb0b96fb3d_study,0.018033,0.814800,0.137384,0.028062


In [11]:
df_result['negative'] = df_result.groupby(['study_id'])['negative'].transform(lambda x: np.max(x))
df_result['typical'] = df_result.groupby(['study_id'])['typical'].transform(lambda x: np.max(x))
df_result['indeterminate'] = df_result.groupby(['study_id'])['indeterminate'].transform(lambda x: np.max(x))
df_result['atypical'] = df_result.groupby(['study_id'])['atypical'].transform(lambda x: np.max(x))

In [12]:
df_result = df_result[['study_id', 'negative', 'typical', 'indeterminate', 'atypical']]

In [13]:
df_result[df_result['study_id']=='00f9e183938e_study']

,study_id,negative,typical,indeterminate,atypical
2641,00f9e183938e_study,0.464355,0.154604,0.207948,0.140807
2642,00f9e183938e_study,0.464355,0.154604,0.207948,0.140807


In [14]:
df_result = df_result.drop_duplicates()
df_result

,study_id,negative,typical,indeterminate,atypical
0,5776db0cec75_study,0.011603,0.875717,0.099689,0.013051
1,ff0879eb20ed_study,0.156081,0.431505,0.259723,0.112396
2,9d514ce429a7_study,0.010057,0.874840,0.101618,0.015602
3,28dddc8559b2_study,0.719496,0.100763,0.137856,0.043034
4,dfd9fdd85a3e_study,0.023360,0.794622,0.151826,0.028687
...,...,...,...,...,...
6329,7e6c68462e06_study,0.081923,0.404266,0.378179,0.130869
6330,8332bdaddb6e_study,0.837703,0.062449,0.078890,0.019056
6331,7eed9af03814_study,0.528679,0.165514,0.213177,0.080163
6332,a0cb0b96fb3d_study,0.018033,0.814800,0.137384,0.028062


In [15]:
def prepare_data(df):
    
    df['negative'] = df['negative'].astype(str)
    df['typical'] = df['typical'].astype(str)
    df['indeterminate'] = df['indeterminate'].astype(str)
    df['atypical'] = df['atypical'].astype(str)
    
    df.loc[:, 'negative'] = 'negative ' + df['negative'] + ' 0 0 1 1 '
    df.loc[:, 'typical'] = 'typical ' + df['typical'] + ' 0 0 1 1 '
    df.loc[:, 'indeterminate'] = 'indeterminate ' + df['indeterminate'] + ' 0 0 1 1 '
    df.loc[:, 'atypical'] = 'atypical ' + df['atypical'] + ' 0 0 1 1'
    df['PredictionString'] = df['negative'] + df['typical'] + df['indeterminate'] + df['atypical']
    df = df[['study_id', 'PredictionString']]
    
    return df

In [16]:
df_result = prepare_data(df_result)

In [17]:
df_result

,study_id,PredictionString
0,5776db0cec75_study,negative 0.011602651 0 0 1 1 typical 0.8757165...
1,ff0879eb20ed_study,negative 0.1560807 0 0 1 1 typical 0.4315046 0...
2,9d514ce429a7_study,negative 0.010057206 0 0 1 1 typical 0.8748395...
3,28dddc8559b2_study,negative 0.7194962 0 0 1 1 typical 0.10076337 ...
4,dfd9fdd85a3e_study,negative 0.023359958 0 0 1 1 typical 0.7946223...
...,...,...
6329,7e6c68462e06_study,negative 0.081922926 0 0 1 1 typical 0.4042661...
6330,8332bdaddb6e_study,negative 0.83770305 0 0 1 1 typical 0.06244928...
6331,7eed9af03814_study,negative 0.52867925 0 0 1 1 typical 0.16551419...
6332,a0cb0b96fb3d_study,negative 0.018033115 0 0 1 1 typical 0.8148004...
